### **Step 1: Install All the Required Packages**

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip uninstall datasets -y
!pip install datasets

### **Step 2: Import All the Required Libraries**

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#### **How to fine tune Llama 2**

### **Step 3**

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "sasvata/MOM-Summary"

# Fine-tuned model name
new_model = "Llama2-7b-MOM-Summary-Finetuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 4

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
#max_steps = -1
max_steps = 250

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio= 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = "auto"

### **Step 4:Load everything and start the fine-tuning process**

1. First of all, we want to load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multiple datasets, etc.


2. Then, we’re configuring bitsandbytes for 4-bit quantization.


3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.


4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [ ]:
# Load dataset (you can process it here)
#dataset = load_dataset(dataset_name, split="train")


dataset = load_dataset(dataset_name, split="train")
dataset = dataset.to_pandas()
dataset['text'] = dataset.apply(lambda x: generate_training_prompt(x['sentences'],x['summary']),axis=1)
dataset = Dataset.from_pandas(dataset)
'''
dataset = pd.read_csv(dataset_name)
dataset = Dataset.from_pandas(dataset)
'''

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

In [6]:
# Save trained model
trainer.model.save_pretrained(new_model)

### **Step 5:Testing the model and validate the response before merging with base model.**

In [7]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()

def generate_prompt(
    Transcript: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{Transcript.strip()}

### Response:
""".strip()

In [8]:
test_transcript = """
{
  "date": "2024-03-05",
  "time": "09:30 AM",
  "attendess": ["Project Manager", "Developer 1", "Developer 2", "QA Engineer"],
  "transcript": [
    "Project Manager: Good morning everyone. Let's kick off our weekly project status meeting. Developer 1, could you please update us on your progress with the feature implementation?
    Developer 1: Sure. I have completed the frontend development for the user authentication module and I'm currently working on integrating it with the backend.
    Project Manager: Excellent. Developer 2, how about you?
    Developer 2: I've been working on optimizing the database queries for better performance. I have managed to reduce the query execution time by 30%.
    Project Manager: That's great progress. QA Engineer, what's the status on the testing front?
    QA Engineer: I've conducted extensive testing on the latest build. So far, I haven't encountered any major issues. However, I'll continue with regression testing to ensure the stability of the system.
    Project Manager: Wonderful. Any roadblocks or challenges anyone is facing?
    Developer 1: I'm facing some challenges with the authentication token generation. The tokens are not being generated correctly.
    Project Manager: Let's allocate some time after the meeting to discuss this further and find a resolution.
    Developer 2: I'm experiencing difficulties with integrating a third-party library for data encryption. The documentation seems outdated.
    Project Manager: I'll assign some resources to assist you with that. QA Engineer, any obstacles on your end?
    QA Engineer: No blockers at the moment. Everything seems to be progressing smoothly from a testing perspective.
    Project Manager: That's good to hear. Keep up the good work, everyone. Let's aim to resolve the issues raised and continue moving forward with our tasks.
    Developer 1: Actually, I have one more thing to add. While working on the integration, I noticed a potential security vulnerability in the authentication module.
    Project Manager: Oh, that's concerning. Can you provide more details?
    Developer 1: It seems that the token expiration time is not properly enforced, which could lead to session hijacking attacks.
    Project Manager: Thank you for bringing this to our attention. Let's prioritize fixing this issue immediately.
    Developer 2: I can assist with reviewing the code and implementing the necessary changes to address the vulnerability.
    Project Manager: Great teamwork. QA Engineer, please ensure to include thorough security testing as part of your regression testing.
    QA Engineer: Understood. I'll make sure to focus on security testing and provide detailed feedback on any vulnerabilities found.
    Project Manager: Perfect. Let's make sure to address this issue promptly and maintain the integrity of our system. Anything else before we conclude?
    Developer 1: No, that's all from my side.
    Developer 2: Likewise, I don't have anything else to add.
    QA Engineer: All good here as well.
    Project Manager: Excellent. Thank you all for your contributions and dedication. Let's continue our efforts to deliver a robust and secure product. Meeting adjourned."
    ]
    }
"""
test_text = generate_prompt(test_transcript)

In [ ]:
print(test_text)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,return_full_text=False,max_length=1000)
result = pipe(test_text)
print(result)

In [ ]:
# Empty VRAM
#del model
#del pipe
del trainer
import gc
gc.collect()
gc.collect()

You can train a Llama 2 model on the entire dataset using [mlabonne/guanaco-llama2](https://huggingface.co/datasets/mlabonne/guanaco-llama2)

### **Step 6: Store New Llama2 Model (Llama-2-7b-finetune)**

How can we store our new Llama-2-7b-finetune model now? We need to merge the weights from LoRA with the base model. Unfortunately, as far as I know, there is no straightforward way to do it: we need to reload the base model in FP16 precision and use the peft library to merge everything.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#**Step 7: Push Model to Hugging Face Hub**

Our weights are merged and we reloaded the tokenizer. We can now push everything to the Hugging Face Hub to save our model.

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#!huggingface-cli login

model.push_to_hub(new_model, check_pr=True)

tokenizer.push_to_hub(new_model,check_pr=True)


You can now use this model for inference by loading it like any other Llama 2 model from the Hub.